# Example for ACE data processing

Jiwoo Lee, Shiheng Duan, Peter Gleckler


Install [CMOR](https://github.com/PCMDI/cmor) if needed following this [instruction](https://anaconda.org/conda-forge/cmor).

In [1]:
"""
conda install conda-forge::cmor
"""

'\nconda install conda-forge::cmor\n'

## Prepare example ACE ouput file 

In [2]:
!ls

cmor_example.ipynb  output_ERA5_small_1


In [3]:
ace_output = 'output_ERA5_small_1/small_sample.nc'

Data quick check

In [4]:
import xarray as xa
ds = xa.open_dataset(ace_output)
ds

<xarray.Dataset> Size: 5GB
Dimensions:                                            (time: 300, sample: 1,
                                                        lat: 180, lon: 360)
Coordinates:
  * time                                               (time) timedelta64[ns] 2kB ...
    init_time                                          (sample) datetime64[ns] 8B ...
    valid_time                                         (sample, time) datetime64[ns] 2kB ...
  * lat                                                (lat) float32 720B -89...
  * lon                                                (lon) float32 1kB 0.5 ...
Dimensions without coordinates: sample
Data variables: (12/59)
    northward_wind_5                                   (sample, time, lat, lon) float32 78MB ...
    total_water_path_budget_residual                   (sample, time, lat, lon) float32 78MB ...
    air_temperature_3                                  (sample, time, lat, lon) float32 78MB ...
    PRATEsfc                                           (sample, time, lat, lon) float32 78MB ...
    total_water_path                                   (sample, time, lat, lon) float32 78MB ...
    air_temperature_6                                  (sample, time, lat, lon) float32 78MB ...
    ...                                                 ...
    UGRD10m                                            (sample, time, lat, lon) float32 78MB ...
    surface_pressure_due_to_dry_air                    (sample, time, lat, lon) float32 78MB ...
    net_energy_flux_toa_into_atmosphere                (sample, time, lat, lon) float32 78MB ...
    surface_temperature                                (sample, time, lat, lon) float32 78MB ...
    air_temperature_0                                  (sample, time, lat, lon) float32 78MB ...
    eastward_wind_2                                    (sample, time, lat, lon) float32 78MB ...

In [5]:
ds.close()

## Extract needed variable

Supporting functions to process ACE output

In [6]:
import numpy as np
import xarray as xa
from tqdm import tqdm


def hybrid_to_pressure_midpoints(aks, bks, surface_pressure):
    # Calculate pressure at the boundaries first
    bks_da = xa.DataArray(bks, dims=("level",), name="bks")
    aks_da = xa.DataArray(aks, dims=("level",), name="aks")

    # Expand `ps` to include the 'level' dimension
    ps_expanded = surface_pressure.expand_dims(level=bks_da.level)

    # Perform the multiplication
    pressure_boundaries = ps_expanded * bks_da + aks_da
    # Calculate pressure at layer midpoints by averaging adjacent boundaries
    pressure_midpoints = (pressure_boundaries[:-1].data + pressure_boundaries[1:].data) / 2.0
    print(pressure_midpoints.shape, ' ', surface_pressure.shape)
    pressure_midpoints = xa.DataArray(pressure_midpoints, dims=["level", "time", "lat", "lon"],
        coords={"level": np.arange(len(aks)-1), "time": surface_pressure.time, 
                "lat": surface_pressure.lat, "lon": surface_pressure.lon}
    )
    return pressure_midpoints, pressure_boundaries


def interpolate_to_pressure_levels(temperature, pressure_levels, target_pressures):
    # Convert target pressures to Pa
    target_pressures = np.array(target_pressures) * 100  # Convert hPa to Pa
    log_target_pressures = np.log(target_pressures)
    
    temperature = temperature.chunk({'level':-1})
    interpolated_temperature = xa.apply_ufunc(
        np.interp,
        log_target_pressures,               # The pressure levels we want to interpolate to (log space)
        np.log(pressure_levels),         # The log of the pressure at midpoints (source log pressures)
        temperature,                    # Temperature data at the midpoints
        input_core_dims=[["plev"], ["level"], ["level"]],  # Interpolation happens along "level"
        output_core_dims=[["plev"]],  # Output will have "pressure" as a new dimension
        vectorize=True,                 # Apply the function to each (time, lat, lon) point
        dask="parallelized",            # Enable parallelization with Dask if needed
        output_dtypes=[temperature.dtype]  # Ensure the correct output dtype
    )
    interpolated_temperature = interpolated_temperature.assign_coords({"plev": target_pressures})
    return interpolated_temperature

In [7]:
variables = ['T', 'U', 'V']
start_year = 1979
end_year = 1979
print('variable:', variables)

# Load hybrid coefficients
ori_data = xa.open_dataset('output_ERA5_small_1/forcing_1979.nc')
print("Hybrid coefficients loaded:", list(ori_data.variables.keys()))

aks = np.array([ori_data[f'ak_{i}'].data for i in range(9)])
bks = np.array([ori_data[f'bk_{i}'].data for i in range(9)])
del ori_data  # Free memory

# Load full dataset
data = xa.open_dataset(ace_output, chunks='auto')
data = data.isel(sample=0)
ps = data['PRESsfc']
print('Surface pressure loaded')

# Define target pressure levels
pnew = [1000.0, 850.0, 500.0, 200.0]

# Process year by year
years = np.unique(data['valid_time.year'].values)

if not isinstance(ps['valid_time'].values[0], np.datetime64):
    ps['valid_time'] = xa.decode_cf(ps['valid_time'])

ps = ps.assign_coords(valid_time=("time", ps["valid_time"].values))
ps = ps.set_index(time="valid_time")
#print(ps)

for variable in variables:
    print(f"--- Process {variable} in the data year by year ---")
    for year in tqdm(range(start_year, end_year+1)):
        # Extract data for the current year
        ps_year = ps.sel(time=str(year))
        ps_year = ps_year.resample(time='1D').mean()
        # Calculate pressure midpoints
        pressure_midpoints, _ = hybrid_to_pressure_midpoints(aks, bks, ps_year)
        print('pressure calculated')
        # Extract and resample variable levels for the current year
        var_levels = []
        for i in range(8):
            var_key = f'air_temperature_{i}' if variable == 'T' else (
                f'eastward_wind_{i}' if variable == 'U' else f'northward_wind_{i}'
            )
            var_year = data[var_key]
            var_year = var_year.assign_coords(valid_time=("time", var_year["valid_time"].values))
            var_year = var_year.set_index(time="valid_time")
            # print(var_year)
            var_year = var_year.sel(time=str(year)).resample(time="1D").mean()
            # var_year = var_year.reindex(time=full_time_range)
            var_levels.append(var_year)

        # Combine levels
        var_combined = xa.concat(var_levels, dim="level")
        print(var_combined.shape)
        print('var combined')
        del var_levels  # Free memory

        # Interpolate to pressure levels
        print(f"Interpolating year {year}")
        var_new = interpolate_to_pressure_levels(var_combined, pressure_midpoints, pnew)
        print('interpolation done')
        #print(var_new)

        # Write the whole year to NetCDF
        output_nc_path = f'output_ERA5_small_1/interpolated_{variable}_{year}.nc'
        var_new.to_netcdf(output_nc_path)
        print(f"Saved year {year} to NetCDF")

variable: ['T', 'U', 'V']
Hybrid coefficients loaded: ['latitude', 'longitude', 'time', 'surface_temperature', 'HGTsfc', 'DSWRFtoa', 'ocean_fraction', 'land_fraction', 'sea_ice_fraction', 'ak_0', 'ak_1', 'ak_2', 'ak_3', 'ak_4', 'ak_5', 'ak_6', 'ak_7', 'ak_8', 'bk_0', 'bk_1', 'bk_2', 'bk_3', 'bk_4', 'bk_5', 'bk_6', 'bk_7', 'bk_8', 'global_mean_co2']
Surface pressure loaded
--- Process T in the data year by year


  0%|          | 0/1 [00:00<?, ?it/s]

(8, 76, 180, 360)   (76, 180, 360)
pressure calculated
(8, 76, 180, 360)
var combined
Interpolating year 1979
interpolation done


100%|██████████| 1/1 [01:10<00:00, 70.28s/it]


Saved year 1979 to NetCDF
--- Process U in the data year by year


...

Saved year 1979 to NetCDF
--- Process V in the data year by year


  0%|          | 0/1 [00:00<?, ?it/s]

(8, 76, 180, 360)   (76, 180, 360)
pressure calculated
(8, 76, 180, 360)
var combined
Interpolating year 1979
interpolation done


100%|██████████| 1/1 [01:03<00:00, 63.24s/it]

Saved year 1979 to NetCDF


In [10]:
!ls output_ERA5_small_1/interpolated_*.nc

output_ERA5_small_1/interpolated_T_1979.nc
output_ERA5_small_1/interpolated_U_1979.nc
output_ERA5_small_1/interpolated_V_1979.nc


## CMORize

In [11]:
ds = xa.open_dataset('output_ERA5_small_1/interpolated_T_1979.nc')

In [12]:
ds

<xarray.Dataset> Size: 79MB
Dimensions:                        (time: 76, lat: 180, lon: 360, plev: 4)
Coordinates:
  * time                           (time) datetime64[ns] 608B 1979-02-01 ... ...
  * lat                            (lat) float32 720B -89.24 -88.25 ... 89.24
  * lon                            (lon) float32 1kB 0.5 1.5 2.5 ... 358.5 359.5
    init_time                      datetime64[ns] 8B ...
  * plev                           (plev) float64 32B 1e+05 8.5e+04 5e+04 2e+04
Data variables:
    __xarray_dataarray_variable__  (time, lat, lon, plev) float32 79MB ...